In [1]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import re

global spark

# SparkContext.setSystemProperty('spark.executor.memory', '20g')

spark = (pyspark.sql.SparkSession
    .builder
    .appName("phenodigm_parser")
    .config("spark.executor.memory", '10g')
     .config("spark.driver.bindAddress", "localhost")
    .config("spark.driver.memory", '10g')
    .getOrCreate()
)

#   


print('Spark version: ', spark.version)

Spark version:  3.0.0


In [5]:
df = spark.createDataFrame(
    [
        (1, "foo", 2, "rs16245"), 
        (2, "bar", 12, "rs2344"),
        (2, "bar", 212, "rs232983"), 
        (2, "bar", 1, "rs20398"),
        (3, "cica", 12, "rs239"), 
        (4, "mica", 12, "rs12235")
    ],
    ["id", "label", "value", "rsid"]  # add your column names here
)

df.printSchema()

df.show()

root
 |-- id: long (nullable = true)
 |-- label: string (nullable = true)
 |-- value: long (nullable = true)
 |-- rsid: string (nullable = true)

+---+-----+-----+--------+
| id|label|value|    rsid|
+---+-----+-----+--------+
|  1|  foo|    2| rs16245|
|  2|  bar|   12|  rs2344|
|  2|  bar|  212|rs232983|
|  2|  bar|    1| rs20398|
|  3| cica|   12|   rs239|
|  4| mica|   12| rs12235|
+---+-----+-----+--------+



In [19]:
unique_fields = ['id','label']

(
    df
    .reduceByKey(unique_fields)
    .show()
)

AttributeError: 'DataFrame' object has no attribute 'reduceByKey'

In [18]:
data_file = spark.parallelize([
   (u'u1', u's1', 20), 
   (u'u1', u's2', 5),
   (u'u2', u's3', 5), 
   (u'u2', u's2', 10)])

max_by_group = (data_file
  .map(lambda x: (x[0], x))  # Convert to PairwiseRD
  # Take maximum of the passed arguments by the last element (key)
  # equivalent to:
  # lambda x, y: x if x[-1] > y[-1] else y
  .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1])) 
  .values()) # Drop keys

max_by_group.collect()
## [('u2', 's2', 10), ('u1', 's1', 20)]

AttributeError: 'SparkSession' object has no attribute 'parallelize'

AttributeError: 'PipelinedRDD' object has no attribute 'show'